## Exploratory Data Analysis – Feature Distributions by Failure Class


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Convert Spark DataFrame → Pandas for visualization
# (sample 10% of the dataset to avoid memory issues on large data)
pdf = features_with_failure.select(
    "Motor_current_avg", 
    "Oil_temp_avg", 
    "DV_pressure_avg", 
    "failure"
).sample(fraction=0.1, seed=42).toPandas()

# Plot distributions of key features, split by failure vs. non-failure
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, col in zip(axes, ["Motor_current_avg", "Oil_temp_avg", "DV_pressure_avg"]):
    # Histogram for normal operation
    pdf[pdf["failure"]==0][col].hist(ax=ax, bins=50, alpha=0.5, label="No Failure")
    # Histogram for failure cases
    pdf[pdf["failure"]==1][col].hist(ax=ax, bins=50, alpha=0.5, label="Failure")
    ax.set_title(f"{col} distribution")
    ax.legend()

plt.show()


## Exploratory Data Analysis – Boxplots of Features by Failure


In [ ]:
import seaborn as sns

# Boxplots to visualize feature distributions grouped by failure vs. non-failure.
# Complements histograms by showing median, quartiles, and outliers.
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, col in zip(axes, ["Motor_current_avg", "Oil_temp_avg", "DV_pressure_avg"]):
    sns.boxplot(x="failure", y=col, data=pdf, ax=ax)
    ax.set_title(f"{col} by failure")

plt.show()


## Exploratory Data Analysis – Sensor Trends Around Failures


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 1. Select a few distinct failure events (timestamps)
failure_times = (
    features_with_failure
    .filter(features_with_failure.failure == 1)
    .select("timestamp_hour")
    .distinct()
    .limit(3)  # sample 3 failures to visualize
    .toPandas()["timestamp_hour"]
)

# 2. Convert Spark DataFrame → Pandas for time series visualization
pdf = features_with_failure.toPandas()

# 3. Normalize signals (z-score) for comparability across features
for col in ["Motor_current_avg", "Oil_temp_avg", "DV_pressure_avg"]:
    pdf[col + "_norm"] = (pdf[col] - pdf[col].mean()) / pdf[col].std()

# 4. Plot trends around each sampled failure event
for ft in failure_times:
    # Create a 24-hour window centered around the failure (±12h)
    window = pdf[
        (pdf["timestamp_hour"] >= ft - pd.Timedelta(hours=12)) &
        (pdf["timestamp_hour"] <= ft + pd.Timedelta(hours=12))
    ]
    
    # Aggregate to hourly means (reduce noise in high-frequency signals)
    window_grouped = (
        window.groupby("timestamp_hour")[[
            "Motor_current_avg_norm", 
            "Oil_temp_avg_norm", 
            "DV_pressure_avg_norm"
        ]].mean().reset_index()
    )
    
    # Time series plot
    plt.figure(figsize=(12,6))
    plt.plot(window_grouped["timestamp_hour"], window_grouped["Motor_current_avg_norm"], 
             label="Motor Current (norm)", color="blue")
    plt.plot(window_grouped["timestamp_hour"], window_grouped["Oil_temp_avg_norm"], 
             label="Oil Temp (norm)", color="orange")
    plt.plot(window_grouped["timestamp_hour"], window_grouped["DV_pressure_avg_norm"], 
             label="DV Pressure (norm)", color="green")
    
    # Mark the failure event
    plt.axvline(ft, color="red", linestyle="--", label="Failure Event")
    plt.title(f"Normalized Sensor Trends Around Failure at {ft}")
    plt.xlabel("Time")
    plt.ylabel("Normalized Sensor Values (z-score)")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.show()


## Exploratory Data Analysis – Aggregated Trends Leading to Failure


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Convert Spark DataFrame → Pandas for plotting
pdf = features_with_failure.toPandas()
pdf['timestamp_hour'] = pd.to_datetime(pdf['timestamp_hour'])

# 2. Identify failure event timestamps
failure_times = pdf[pdf['failure'] == 1]['timestamp_hour']

# 3. Collect pre-failure windows (12h before each failure)
records = []
window = 12  # hours lookback

for ft in failure_times:
    subset = pdf[
        (pdf['timestamp_hour'] >= ft - pd.Timedelta(hours=window)) &
        (pdf['timestamp_hour'] <= ft)
    ].copy()
    # Add relative time axis ("hours to failure")
    subset['hours_to_failure'] = (subset['timestamp_hour'] - ft).dt.total_seconds() / 3600
    records.append(subset)

# 4. Combine all pre-failure windows into one aligned dataset
aligned = pd.concat(records)

# 5. Aggregate mean + std for sensor values across failures
agg = (
    aligned
    .groupby('hours_to_failure')[['Motor_current_avg','Oil_temp_avg','DV_pressure_avg']]
    .agg(['mean','std'])
)

# 6. Normalize (z-score across time) for comparability
norm = (agg - agg.mean()) / agg.std()

# 7. Plot aggregated normalized trends with ±1 std shading
plt.figure(figsize=(12,6))

for sensor, color in zip(['Motor_current_avg','Oil_temp_avg','DV_pressure_avg'],
                         ['blue','orange','green']):
    mean = norm[sensor]['mean']
    std = norm[sensor]['std']
    plt.plot(mean.index, mean.values, label=f"{sensor} (norm)", color=color)
    plt.fill_between(mean.index, mean-std, mean+std, color=color, alpha=0.2)

plt.axvline(0, color="red", linestyle="--", label="Failure Event")
plt.title("Normalized Sensor Trends Before Failure (Mean ± Std)")
plt.xlabel("Hours to Failure")
plt.ylabel("Normalized Sensor Values (z-score)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()


## Exploratory Data Analysis – Feature Distributions & Correlations


In [ ]:
# --- Step 3: EDA (Part 2: Distributions + Correlations) ---
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Convert Spark DataFrame → Pandas for plotting
pdf = df_final.toPandas()

# 2. Identify numeric columns (ignore categorical / string cols)
num_cols = [c for c in pdf.columns if pd.api.types.is_numeric_dtype(pdf[c])]

# --- Histograms for numeric sensor features ---
# Shows distributions (shape, skew, outliers) for first 12 numeric columns
plt.figure(figsize=(16, 10))
for i, col in enumerate(num_cols[:12]):  # limit to 12 for readability
    plt.subplot(3, 4, i+1)
    sns.histplot(pdf[col].dropna(), bins=40, kde=True, color="steelblue")
    plt.title(col, fontsize=9)
    plt.tight_layout()
plt.suptitle("Sensor Distributions (sampled)", fontsize=14, weight="bold")
plt.show()

# --- Correlation heatmap ---
# Highlights relationships between engineered features
# Useful for detecting multicollinearity and redundant features
plt.figure(figsize=(14, 10))
corr = pdf[num_cols].corr()
sns.heatmap(corr, cmap="coolwarm", center=0, annot=False, cbar=True)
plt.title("Correlation Heatmap of Engineered Features", fontsize=14, weight="bold")
plt.show()


## Exploratory Data Analysis – Correlations & Boxplots of Key Sensors


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Convert Spark → Pandas for easier plotting
pdf = df_final.toPandas()

# 2. Filter numeric columns only
num_cols = [c for c in pdf.columns if pd.api.types.is_numeric_dtype(pdf[c])]
pdf_num = pdf[num_cols].dropna()

# -----------------------------
# Correlation Heatmap
# -----------------------------
# Shows linear relationships between numeric features
plt.figure(figsize=(14, 10))
corr = pdf_num.corr()
sns.heatmap(corr, cmap="coolwarm", center=0, square=True, cbar_kws={"shrink": 0.8})
plt.title("Sensor Correlation Heatmap", fontsize=16, weight="bold")
plt.show()

# -----------------------------
# Boxplots for Key Sensor Signals
# -----------------------------
# Compares distributions + outliers for a selected subset of features
sample_sensors = [
    "Motor_current_avg",
    "Oil_temperature_avg",
    "DV_pressure_avg",
    "gpsSpeed_avg"
]

plt.figure(figsize=(12, 6))
pdf_melt = pdf[sample_sensors].melt(var_name="Sensor", value_name="Value")
sns.boxplot(x="Sensor", y="Value", data=pdf_melt)
plt.title("Boxplots of Key Sensor Signals", fontsize=16, weight="bold")
plt.xticks(rotation=30)
plt.show()


## Dataset Dimensions Check


In [ ]:
# Rows and columns before dropping non-feature columns
print("Full dataset shape:", pdf.shape)

# Rows and columns actually used for modeling (X only)
print("Feature matrix shape:", X.shape)

# Target vector length
print("Target vector length:", y.shape)

# Raw data dimensions
print("Raw row count:", iot_df.count())
print("Raw column count:", len(iot_df.columns))



## Model Training & Evaluation (RandomForest, XGBoost)


In [ ]:
# =========================================================
# Libraries & Setup
# =========================================================
# Core data handling & visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Machine learning models & metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Ensure consistent plotting style
plt.rcParams["figure.dpi"] = 120

# Backwards-compatible RMSE function for sklearn
try:
    from sklearn.metrics import root_mean_squared_error as rmse_fn
except Exception:
    def rmse_fn(y_true, y_pred):
        return mean_squared_error(y_true, y_pred, squared=False)


# =========================================================
# 1) Prepare Training Data
# =========================================================
# Keep only rows with valid Remaining Useful Life (RUL) labels
train_df = df_final.filter(df_final["RUL_minutes"].isNotNull())
pdf = train_df.toPandas()

# (Optional) Add cyclic features for time-of-day
if "timestamp_bin" in pdf.columns:
    tod_min = pdf["timestamp_bin"].dt.hour * 60 + pdf["timestamp_bin"].dt.minute
    pdf["tod_sin"] = np.sin(2 * np.pi * tod_min / 1440.0)
    pdf["tod_cos"] = np.cos(2 * np.pi * tod_min / 1440.0)

# Define target variable and drop metadata columns
drop_cols = [
    "timestamp_bin", 
    "failure", 
    "next_failure_time", 
    "last_failure_time", 
    "minutes_since_last_failure", 
    "RUL_minutes"
]
y = pdf["RUL_minutes"]
num_cols = [c for c in pdf.columns if c not in drop_cols and pd.api.types.is_numeric_dtype(pdf[c])]
X = pdf[num_cols].copy()

# Train/test split without shuffling (time-ordered)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Handle NaNs/infs by replacing with training medians
train_medians = X_train.median(numeric_only=True)
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(train_medians)
X_test  = X_test.replace([np.inf, -np.inf], np.nan).fillna(train_medians)

print(f"Feature matrix shape: {X.shape}  |  Train: {X_train.shape}  Test: {X_test.shape}")
print(f"Target length: {y.shape}")


# =========================================================
# 2) Train Models
# =========================================================
# Random Forest Regressor (simplified config for speed)
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=12,
    n_jobs=-1,
    random_state=42
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# XGBoost Regressor (faster training config)
xgb_model = xgb.XGBRegressor(
    objective="reg:squarederror",
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",
    n_jobs=-1,
    random_state=42
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)


# =========================================================
# 3) Evaluate Models
# =========================================================
def evaluate(name, y_true, y_pred):
    """Compute MAE and RMSE for a given model’s predictions"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = rmse_fn(y_true, y_pred)
    print(f"{name:12s} → MAE: {mae:.3f} | RMSE: {rmse:.3f}")
    return mae, rmse

mae_rf,  rmse_rf  = evaluate("RandomForest", y_test, y_pred_rf)
mae_xgb, rmse_xgb = evaluate("XGBoost",      y_test, y_pred_xgb)


# =========================================================
# 4) Visualizations
# =========================================================
plt.figure(figsize=(13,5))

# (a) Predicted vs Actual
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred_rf, alpha=0.4, label="RF")
plt.scatter(y_test, y_pred_xgb, alpha=0.4, label="XGB")
mn, mx = float(y_test.min()), float(y_test.max())
plt.plot([mn, mx], [mn, mx], "k--", linewidth=1)
plt.xlabel("Actual RUL (min)")
plt.ylabel("Predicted RUL (min)")
plt.title("Predicted vs Actual")
plt.legend()

# (b) Error Distribution
plt.subplot(1, 3, 2)
plt.hist(y_pred_rf - y_test, bins=40, alpha=0.6, label="RF")
plt.hist(y_pred_xgb - y_test, bins=40, alpha=0.6, label="XGB")
plt.xlabel("Prediction Error (min)")
plt.ylabel("Frequency")
plt.title("Error Distribution")
plt.legend()

# (c) Feature Importance (Top 15, XGBoost)
plt.subplot(1, 3, 3)
imp = pd.Series(xgb_model.feature_importances_, index=X_train.columns)
topk = imp.sort_values(ascending=False).head(15)[::-1]
plt.barh(topk.index, topk.values)
plt.title("XGBoost Feature Importance (Top 15)")

plt.tight_layout()
plt.show()

# Print top contributing features
rf_imp = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False).head(20)
xgb_imp = imp.sort_values(ascending=False).head(20)

print("\nTop 20 features (RF):")
print(rf_imp)

print("\nTop 20 features (XGB):")
print(xgb_imp)


## LSTM Modeling: Sequence Approach (Aligned with RF/XGB Split)

In [ ]:
# ---- Make LSTM use the exact same split as RF/XGB and add it to comparison ----
import numpy as np, torch, torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# 1) Reconstruct the full, already-imputed matrix & target
X_full = np.vstack([np.asarray(X_train), np.asarray(X_test)])
y_full = np.concatenate([np.asarray(y_train), np.asarray(y_test)])
cut    = len(y_train)                      # same cutoff as RF/XGB
SEQ_LEN = 30

# 2) Build rolling sequences whose TARGET index aligns with the same cutoff
Xs, ys, tgt = [], [], []
for i in range(len(X_full) - SEQ_LEN):
    Xs.append(X_full[i:i+SEQ_LEN])
    ys.append(y_full[i+SEQ_LEN])
    tgt.append(i + SEQ_LEN)
Xs, ys, tgt = np.array(Xs), np.array(ys), np.array(tgt)

mask_test  = tgt >= cut                    # targets on/after the cutoff → test
Xseq_tr, Yseq_tr = Xs[~mask_test], ys[~mask_test]
Xseq_te, Yseq_te = Xs[mask_test], ys[mask_test]

assert len(Yseq_te) == len(y_test), "LSTM test size must match RF/XGB test size"

# 3) Normalize using TRAIN sequences only
mu  = Xseq_tr.mean(axis=(0,1), keepdims=True)
std = Xseq_tr.std(axis=(0,1),  keepdims=True); std[std==0] = 1e-6
Xseq_tr = (Xseq_tr - mu)/std
Xseq_te = (Xseq_te - mu)/std

# 4) Tiny LSTM
class LSTMRegressor(nn.Module):
    def __init__(self, n_features, hidden=64, layers=2):
        super().__init__()
        self.lstm = nn.LSTM(n_features, hidden, num_layers=layers, batch_first=True, dropout=0.2)
        self.fc   = nn.Linear(hidden, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :]).squeeze(-1)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTMRegressor(n_features=Xseq_tr.shape[2]).to(device)
opt   = torch.optim.Adam(model.parameters(), lr=1e-3)
lossf = nn.MSELoss()

train_ds = TensorDataset(torch.tensor(Xseq_tr, dtype=torch.float32),
                         torch.tensor(Yseq_tr, dtype=torch.float32))
train_ld = DataLoader(train_ds, batch_size=256, shuffle=True)

model.train()
for epoch in range(6):  # quick fit
    epoch_loss = 0.0
    for xb, yb in train_ld:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        pred = model(xb)
        loss = lossf(pred, yb)
        loss.backward(); opt.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}: loss={epoch_loss/len(train_ld):.4f}")

# 5) Predict on SAME test span as RF/XGB
model.eval()
with torch.no_grad():
    y_pred_lstm = model(torch.tensor(Xseq_te, dtype=torch.float32, device=device)).cpu().numpy()

# 6) Add to your existing comparison dict & re-plot (reuse your earlier block)
models["LSTM (seq)"] = (Yseq_te, y_pred_lstm)

# Re-run the metrics + plotting cell you used for RF/XGB (no other changes needed).


## LSTM Model Evaluation and Visualization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ============================================================
# 1) Compute metrics safely
# ============================================================
eps = 1e-6
mask = np.isfinite(y_test) & np.isfinite(y_pred)
yt = np.asarray(y_test)[mask]
yp = np.asarray(y_pred)[mask]

errors = yp - yt
abs_err = np.abs(errors)

mae  = mean_absolute_error(yt, yp)
rmse = mean_squared_error(yt, yp, squared=False)
bias = float(np.mean(errors))
smape = 100.0 * np.mean(2.0 * abs_err / (np.abs(yt) + np.abs(yp) + eps))
pct_w5  = 100.0 * np.mean(abs_err <= 5)
pct_w10 = 100.0 * np.mean(abs_err <= 10)

# calibration fit
coef = np.polyfit(yt, yp, deg=1) if yt.size > 1 else [0, 0]
fit_fn = np.poly1d(coef)
r2 = r2_score(yt, yp) if yt.size > 1 else 0.0

# ============================================================
# 2) Plotting
# ============================================================
plt.style.use("default")
fig = plt.figure(figsize=(16, 9))
grid = fig.add_gridspec(3, 4, height_ratios=[0.6, 1.6, 1.6], hspace=0.8, wspace=0.8)

# (A) Metric header table
ax_head = fig.add_subplot(grid[0, :])
ax_head.axis("off")
summary = pd.DataFrame([{
    "Model": "LSTM",
    "MAE (min)": f"{mae:.2f}",
    "RMSE (min)": f"{rmse:.2f}",
    "Bias (min)": f"{bias:.2f}",
    "sMAPE (%)": f"{smape:.1f}",
    "≤5 min": f"{pct_w5:.1f}%",
    "≤10 min": f"{pct_w10:.1f}%"
}])
tbl = ax_head.table(cellText=summary.values, colLabels=summary.columns,
                    cellLoc="center", loc="center")
tbl.auto_set_font_size(False)
tbl.set_fontsize(11)
ax_head.set_title("Accuracy Summary", pad=12, fontsize=16, weight="bold")

# (B) Predicted vs Actual scatter
ax_sc = fig.add_subplot(grid[1:, 0:2])
if yt.size > 0:
    ax_sc.scatter(yt, yp, s=15, alpha=0.5, edgecolors="none")
    mn, mx = float(np.min(yt)), float(np.max(yt))
    ax_sc.plot([mn, mx], [mn, mx], "r--", lw=2, label="Ideal = Predicted")
    if yt.size > 5:
        xx = np.linspace(mn, mx, 100)
        ax_sc.plot(xx, fit_fn(xx), lw=2,
                   label=f"Fit: y={coef[0]:.2f}x+{coef[1]:.2f} (R²={r2:.2f})")
    ax_sc.set_xlabel("Actual RUL (minutes)")
    ax_sc.set_ylabel("Predicted RUL (minutes)")
    ax_sc.set_title("LSTM: Predicted vs Actual")
    ax_sc.grid(True, alpha=0.3)
    ax_sc.legend(loc="upper left")
else:
    ax_sc.text(0.5, 0.5, "No valid test samples", ha="center", va="center")
    ax_sc.set_axis_off()

# (C) Error histogram
ax_err = fig.add_subplot(grid[1:, 2:])
if errors.size > 0:
    ax_err.hist(errors, bins=60, edgecolor="black", alpha=0.7)
    ax_err.axvline(0, color="k", linestyle="--", linewidth=1, label="Zero error")
    ax_err.axvline(errors.mean(), color="tab:red", linestyle="--", linewidth=2,
                   label=f"Mean bias = {errors.mean():.2f} min")
    ax_err.set_title("Prediction Error Distribution (Pred - Actual)")
    ax_err.set_xlabel("Error (minutes)")
    ax_err.set_ylabel("Count")
    ax_err.grid(True, alpha=0.3)
    ax_err.legend()
else:
    ax_err.text(0.5, 0.5, "No errors to display", ha="center", va="center")
    ax_err.set_axis_off()

# (D) Epoch loss inset (if collected)
if "epoch_losses" in globals() and len(epoch_losses) > 0:
    inset = ax_sc.inset_axes([0.55, 0.05, 0.4, 0.35])
    inset.plot(range(1, len(epoch_losses)+1), epoch_losses, marker="o", ms=3)
    inset.set_title("Epoch Loss", fontsize=10)
    inset.set_xlabel("Epoch", fontsize=9)
    inset.set_ylabel("MSE", fontsize=9)
    inset.grid(True, alpha=0.3)
else:
    ax_sc.text(0.02, 0.02, "No epoch loss history recorded",
               transform=ax_sc.transAxes, fontsize=9, color="gray")

plt.show()
